# Evaluation Result

In [2]:
from typing import List, Dict

In [13]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def left_diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li2]
    return li_dif

def right_diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1]
    return li_dif

def extract_polarity(prediction, variable_name):
    polarity = {}
    for pred in prediction:
        polarity[pred[variable_name]] = pred['polarity']
    return polarity

def evaluate_term(prediction: List[Dict], labels: List[Dict]):
    assert len(prediction) == len(labels)
    result = {
        'fp': 0,
        'fn': 0,
        'tp': 0,
        'tn': 0
    }
    for pred, label in zip(prediction, labels):
        # true negative
        if len(pred) == 0 and len(label) == 0:
            result['tn'] = result['tn'] + 1
        
        pred_terms = [p['word'] for p in pred]
        label_terms = [l['term'] for l in label]
        # true positive
        result['tp'] = result['tp'] + len(intersection(pred_terms, label_terms))
        # false positive
        result['fp'] = result['fp'] + len(left_diff(pred_terms, label_terms))
        # false positive
        result['fn'] = result['fn'] + len(right_diff(pred_terms, label_terms))
    return result

def evaluate_polarity(prediction: List[Dict], labels: List[Dict]):
    assert len(prediction) == len(labels)
    result = {
        'fp': 0,
        'fn': 0,
        'tp': 0,
        'tn': 0
    }
    for pred, label in zip(prediction, labels):
        pred_terms = [p['word'] for p in pred]
        label_terms = [l['term'] for l in label]
        pred_polarity = extract_polarity(pred, 'word')
        label_polarity = extract_polarity(label, 'term')
        correct_term_prediction = intersection(pred_terms, label_terms)
        for term in correct_term_prediction:
            # positives
            if pred_polarity[term] == 'positive':
                if pred_polarity[term] == label_polarity[term]:
                    result['tp'] = result['tp'] + 1
                else:
                    result['fp'] = result['fp'] + 1
            else:
                if pred_polarity[term] == label_polarity[term]:
                    result['tn'] = result['tn'] + 1
                else:
                    result['fn'] = result['fn'] + 1
    return result

# def evaluate_term_category(prediction: List[Dict], labels: List[Dict], unique_category: List):
#     assert len(prediction) == len(labels)
#     result = {
#         'fp': 0,
#         'fn': 0,
#         'tp': 0,
#         'tn': 0
#     }
#     for pred, label in zip(prediction, labels):
#         pred_terms = [p['word'] for p in pred]
#         label_terms = [l['term'] for l in label]
#         pred_polarity = extract_polarity(pred, 'word')
#         label_polarity = extract_polarity(label, 'term')
#         correct_term_prediction = intersection(pred_terms, label_terms)
#         for term in correct_term_prediction:
#             # positives
#             if pred_polarity[term] == 'positive':
#                 if pred_polarity[term] == label_polarity[term]:
#                     result['tp'] = result['tp'] + 1
#                 else:
#                     result['fp'] = result['fp'] + 1
#             else:
#                 if pred_polarity[term] == label_polarity[term]:
#                     result['tn'] = result['tn'] + 1
#                 else:
#                     result['fn'] = result['fn'] + 1
#     return result

def evaluate_sentence_category():
    pass

In [14]:
from datasets import load_from_disk
data = load_from_disk('../data_2015/aspect term result/glove embedding')

In [17]:
eval_term = evaluate_term(data['pos_tag'], data['term ground truth'])

In [23]:
(eval_term['tp'] + eval_term['tn'])/sum(eval_term.values()) * 100

26.060968320382543

In [20]:
eval_polarity = evaluate_polarity(data['pos_tag'], data['term ground truth'])

In [24]:
(eval_polarity['tp'] + eval_polarity['tn'])/sum(eval_polarity.values()) * 100

91.62844036697247